In [ ]:
# 기사 목록

from selenium import webdriver
#from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
#from selenium.webdriver.support.ui import WebDriverWait
#from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
import urllib.parse

# 크롬 드라이버 옵션 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 필요시 활성화
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--blink-settings=imagesEnabled=false")  # 이미지 로드 비활성화
chrome_options.add_argument("--disable-extensions")  # 확장 프로그램 비활성화
chrome_options.add_argument("--disable-gpu")  # GPU 가속 비활성화

# Keywords to search
keywords = [
    "검색어",
]
start_date = "20240601"
end_date = "20240831"

seen_urls = set()

# 웹 드라이버 초기화
driver = webdriver.Chrome(options=chrome_options)

def fetch_search_results(keyword, start_date, end_date):
    base_url = "https://www.knnews.co.kr/search_new/search.php?sfrom=knnews_blog&stype=all&otype=date&category=&interval="
    results = []
    page = 1

    while True:
        print(f"{keyword} {page}")
        search_url = f"{base_url}{start_date}~{end_date}&keyword={urllib.parse.quote(keyword)}&hkeyword=&cpage={page}&optionDetail=&redate=all&bookdetail="
        driver.get(search_url)
        
        try:
            # 페이지 로드 완료 대기
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')

            # 기사 목록 가져오기
            articles_section = soup.find("div", class_="se_cont1")
            if articles_section is None:
                print(f"페이지 {page}에서 기사 섹션을 찾을 수 없습니다.")
                break

            articles = articles_section.find_all("li")
            if not articles:
                break

            # 기사 정보 추출
            for item in articles:
                try:

                    # 제목과 링크 추출
                    title_tag = item.find("a", class_="result_item_title")
                    if title_tag:
                        title = title_tag.text.strip()
                        href = title_tag["href"].replace("../../", "")
                        url = "https://www.knnews.co.kr/" + href

                    else:
                        title = ""
                        url = ""

                    # 중복 URL 체크
                    if url in seen_urls:
                        continue  # 이미 처리된 URL은 무시
                    seen_urls.add(url)  # 새로운 URL 저장

                    # 기자명과 날짜 추출
                    writer_date = item.find("span", class_="se_cont1_date02") or item.find("span", class_="se_cont1_date")
                    if writer_date:
                        writer_date_text = writer_date.text.strip()
                        parts = writer_date_text.split("/")
                        if len(parts) == 3:  # se_cont1_date02의 경우
                            writer_part = parts[0]
                            date_part = parts[1].strip() + "/" + parts[2].strip()
                            writer = writer_part.split(":")[-1].strip() if "기자명" in writer_part else ""
                            date = date_part.strip().split()[0]  # 날짜에서 시간 부분 제거
                        elif len(parts) == 2:
                            writer_part, date_part = parts
                            writer = writer_part.split(":")[-1].strip() if "기자명" in writer_part else ""
                            date = date_part.strip().split()[0]  # 날짜에서 시간 부분 제거
                        else:
                            writer = ""
                            date = writer_date_text.strip().split()[0]  # 날짜에서 시간 부분 제거
                    else:
                        writer = ""
                        date = ""


                    # 결과 리스트에 추가
                    results.append({
                        'media': '경남신문', 
                        'title': title, 
                        'date': date, 
                        'section': '',  # 섹션 정보 없음
                        'article_type': '',  # 기사 유형 정보 없음
                        'writer': writer, 
                        'page': '', 
                        'url': url, 
                        'content': ''  # 컨텐츠 정보 없음
                    })
                except Exception as e:
                    print(f"기사 정보 추출 중 오류 발생: {e}")

        except Exception as e:
            print(f"페이지 {page} 로드 중 오류 발생: {e}")
            break

        page += 1  # 다음 페이지로 이동
    
    return results

def save_to_excel(data, filename):
    df = pd.DataFrame(data)
    df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
    df.to_excel(filename, index=False)

all_results = []
try:
    for keyword in keywords:
        results = fetch_search_results(keyword, start_date, end_date)
        all_results.extend(results)

    # 중복된 기사 제거 (url 기준)
    df = pd.DataFrame(all_results)

    # 기사 수 출력
    print(f"Total number of articles: {len(df)}")

    # 중복 제거된 데이터 저장
    save_to_excel(df, f'2경남신문 {start_date}~{end_date}.xlsx')

finally:
    # 드라이버 종료
    driver.quit()


In [ ]:
# 목록 추출

import os
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup

# 1. 출처와 파일명 변수 설정
file_name = '파일명'  # 파일명 설정
base_path = rf'경로'  # 경로 설정

# 2. 임시 파일 경로와 최종 파일 경로 생성
temp_output_path = os.path.join(base_path, f'{file_name}.xlsx')
# 임시 파일 에러를 대비해 본 파일에 덮어쓴다.
output_path = os.path.join(base_path, f'{file_name}_본문.xlsx')  # 최종 파일 경로
error_log_path = os.path.join(base_path, f'{file_name}_error_log.txt')  # 에러 로그 파일 경로

# 3. 파일 경로 생성 및 엑셀 파일 불러오기
input_file = os.path.join(base_path, f'{file_name}.xlsx')
df = pd.read_excel(input_file, engine='openpyxl')

# Selenium 헤드리스 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# ChromeDriver 설정 없이 크롬 브라우저 실행
driver = webdriver.Chrome(options=chrome_options)

# URL에서 본문과 서브 타이틀을 스크랩하는 함수 정의
def get_content_from_url(url):
    try:
        driver.get(url)
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 본문 추출
        article_body = soup.find('li', id="content_li")

        # 서브 타이틀 추출 (h2.h2_tex)
        subtitle = ""
        subtitle_tags = [soup.find('h2', class_='h2_tex'), soup.find('b')]
        subtitle = '\n'.join([tag.get_text(separator='\n', strip=True) for tag in subtitle_tags if tag]) or ""
        [tag.decompose() for tag in subtitle_tags if tag]

        if article_body:
            # 'caption' 클래스를 가진 모든 태그를 제거합니다.
            for caption in article_body.find_all('div', align='center'):
                caption.decompose()

            # 'photo_content' 클래스를 가진 div 태그 제거
            for photo_content in article_body.find_all('div', class_='photo_content'):
                photo_content.decompose()

            content = article_body.get_text(separator='\n', strip=True)
        else:
            content = ""
        return content, subtitle
    except Exception as e:
        print(f"Error fetching URL {url}: {e}")
        return None, None

# 진행 중인 기사 스크래핑 상태를 트래킹할 리스트와 로그 파일
error_log = []
total_articles = len(df)

# 각 행의 URL에서 본문과 서브 타이틀 내용을 스크랩하여 content와 sub-title 컬럼에 저장합니다.
for index, row in df.iterrows():
    print(f"Processing {index + 1}/{total_articles}: {row['url']}")
    content, subtitle = get_content_from_url(row['url'])

    if content and content.startswith('='):
        content = "'" + content

    df.at[index, 'content'] = content
    df.at[index, 'sub-title'] = subtitle  # 서브 타이틀 추가

    # 100개마다 파일 저장
    if (index + 1) % 100 == 0:
        df.to_excel(temp_output_path, index=False)
        print(f"임시 파일을 저장했습니다: {temp_output_path}")

# 웹드라이버 종료
driver.quit()

# 변경된 데이터를 파일로 저장합니다.
df = df[~df['title'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df = df[~df['content'].str.contains('블락비|나인뮤지스', case=False, na=False)]
df.to_excel(output_path, index=False)

print("본문 내용을 성공적으로 스크랩하여 저장했습니다.")

# 에러 로그 파일 저장
if error_log:
    with open(error_log_path, 'w') as f:
        for url in error_log:
            f.write(f"{url}\n")
    print(f"에러 로그가 저장되었습니다: {error_log_path}")
